In [ ]:
import sys
!{sys.executable} -m pip install --user pandas scikit-learn tensorflow --upgrade keras nltk gensim transformers evaluate numpy torch

In [ ]:
import os

import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices("GPU")

In [ ]:
nltk.download('stopwords')

In [ ]:
df = pd.read_csv("data/amazon.csv")
df.info()

In [ ]:
for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_csv(os.path.join(root, filename))
        df = pd.concat([df, temp_df], axis=0, sort=False)

df.info()

In [ ]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.drop(columns=[df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"}, inplace=True)
df.info()

In [ ]:
df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["percentage_price_difference"] = df["absolute_price_difference"] / df["price_before"] * 100
df["price_change_direction"] = np.where(df["absolute_price_difference"] > 0, 1, 0)
df.head()

In [ ]:
df[df["price_change_direction"] == 1].count()

In [ ]:
df[df["price_change_direction"] == 0].count()

In [ ]:
news_df = pd.concat([df["news_text"], df["price_change_direction"]], axis=1)
news_df.head()

In [ ]:
X = news_df["news_text"].values
y = news_df["price_change_direction"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.2, shuffle = True, random_state=0)
print(len(X_train),len(X_test),len(y_train),len(y_test))

X_train = X_train.tolist()

In [ ]:
from transformers import AutoTokenizer,  DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(X_train, padding=True, truncation=True, return_tensors="tf")
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = tf.convert_to_tensor(y_train)  # Label is already an array of 0 and 1

In [ ]:
labels.shape

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))

model.fit(tokenized_data, labels, epochs=15, batch_size=8)

In [ ]:
import evaluate

acc = evaluate.load("accuracy")
# Round the predictions to turn them into "0" or "1" labels

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model)
predictions = trainer.predict(X_test)
predictions.predictions

In [24]:
from transformers import Trainer

trainer = Trainer(model=model)
predictions = trainer.predict(X_test)
predictions.predictions

ImportError: 
Trainer requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFTrainer".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [20]:
test_preds = np.round(model.predict(X_test))
test_labels = y_test

print("Test accuracy is : ", acc.compute(predictions = test_preds, references = test_labels))

ValueError: in user code:

    File "/home/alexa/.local/lib/python3.9/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/alexa/.local/lib/python3.9/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alexa/.local/lib/python3.9/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/alexa/.local/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/alexa/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file0inq40is.py", line 36, in tf__run_call_with_unpacked_inputs
        retval_ = ag__.converted_call(ag__.ld(f